# Data Wrangling

**Import Packages**

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from datetime import timedelta
from pandas.tseries.offsets import BDay

**Load and Read File**

In [2]:
df_stock = pd.read_csv("../Datasets/prices-split-adjusted.csv")
df_stock.head()

,date,symbol,open,close,low,high,volume
0,2016-01-05,WLTW,123.430000,125.839996,122.309998,126.250000,2163600.0
1,2016-01-06,WLTW,125.239998,119.980003,119.940002,125.540001,2386400.0
2,2016-01-07,WLTW,116.379997,114.949997,114.930000,119.739998,2489500.0
3,2016-01-08,WLTW,115.480003,116.620003,113.500000,117.440002,2006300.0
4,2016-01-11,WLTW,117.010002,114.970001,114.089996,117.330002,1408600.0


In [3]:
fundamentals = pd.read_csv("../Datasets/fundamentals.csv")
fundamentals.head()

,Unnamed: 0,Ticker Symbol,Period Ending,Accounts Payable,Accounts Receivable,Add'l income/expense items,After Tax ROE,Capital Expenditures,Capital Surplus,Cash Ratio,...,Total Current Assets,Total Current Liabilities,Total Equity,Total Liabilities,Total Liabilities & Equity,Total Revenue,Treasury Stock,For Year,Earnings Per Share,Estimated Shares Outstanding
0,0,AAL,2012-12-31,3.068000e+09,-222000000.0,-1.961000e+09,23.0,-1.888000e+09,4.695000e+09,53.0,...,7.072000e+09,9.011000e+09,-7.987000e+09,2.489100e+10,1.690400e+10,2.485500e+10,-367000000.0,2012.0,-5.60,3.350000e+08
1,1,AAL,2013-12-31,4.975000e+09,-93000000.0,-2.723000e+09,67.0,-3.114000e+09,1.059200e+10,75.0,...,1.432300e+10,1.380600e+10,-2.731000e+09,4.500900e+10,4.227800e+10,2.674300e+10,0.0,2013.0,-11.25,1.630222e+08
2,2,AAL,2014-12-31,4.668000e+09,-160000000.0,-1.500000e+08,143.0,-5.311000e+09,1.513500e+10,60.0,...,1.175000e+10,1.340400e+10,2.021000e+09,4.120400e+10,4.322500e+10,4.265000e+10,0.0,2014.0,4.02,7.169154e+08
3,3,AAL,2015-12-31,5.102000e+09,352000000.0,-7.080000e+08,135.0,-6.151000e+09,1.159100e+10,51.0,...,9.985000e+09,1.360500e+10,5.635000e+09,4.278000e+10,4.841500e+10,4.099000e+10,0.0,2015.0,11.39,6.681299e+08
4,4,AAP,2012-12-29,2.409453e+09,-89482000.0,6.000000e+05,32.0,-2.711820e+08,5.202150e+08,23.0,...,3.184200e+09,2.559638e+09,1.210694e+09,3.403120e+09,4.613814e+09,6.205003e+09,-27095000.0,2012.0,5.29,7.328355e+07


We will explore the fundamentals dataset further [below](#fundamentals).

In [4]:
df_stock.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 851264 entries, 0 to 851263
Data columns (total 7 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   date    851264 non-null  object 
 1   symbol  851264 non-null  object 
 2   open    851264 non-null  float64
 3   close   851264 non-null  float64
 4   low     851264 non-null  float64
 5   high    851264 non-null  float64
 6   volume  851264 non-null  float64
dtypes: float64(5), object(2)
memory usage: 45.5+ MB


In [5]:
fundamentals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1781 entries, 0 to 1780
Data columns (total 79 columns):
 #   Column                                               Non-Null Count  Dtype  
---  ------                                               --------------  -----  
 0   Unnamed: 0                                           1781 non-null   int64  
 1   Ticker Symbol                                        1781 non-null   object 
 2   Period Ending                                        1781 non-null   object 
 3   Accounts Payable                                     1781 non-null   float64
 4   Accounts Receivable                                  1781 non-null   float64
 5   Add'l income/expense items                           1781 non-null   float64
 6   After Tax ROE                                        1781 non-null   float64
 7   Capital Expenditures                                 1781 non-null   float64
 8   Capital Surplus                                      1781 non-null  

In [6]:
# change date columns to data type datetime
df_stock['date']=pd.to_datetime(df_stock['date'])
fundamentals['Period Ending'] = pd.to_datetime(fundamentals['Period Ending'])
print(df_stock.dtypes['date'])
print(fundamentals.dtypes['Period Ending'])

datetime64[ns]
datetime64[ns]


Since we know that we are going to have the merge the two datasets on tickers and dates, let's examine the dates columns for both datasets to get an idea of where would be the best place to merge without losing data

In [7]:
df_stock['date'].dt.year.value_counts()

2016    126125
2015    124957
2014    123480
2013    122624
2011    118440
2010    118138
2012    117500
Name: date, dtype: int64

In [8]:
fundamentals['Period Ending'].dt.year.value_counts()

2015    445
2013    439
2014    436
2012    236
2016    220
2017      1
2007      1
2006      1
2004      1
2003      1
Name: Period Ending, dtype: int64

The years 2013, 2014, and 2015 look promising as they have the most data. The other years appear to be missing their fundamentals data for many of their tickers.

Let's look at year 2013 for now and compare the two dataframes' unique stock tickers

In [9]:
stock_2013 = df_stock[df_stock['date'].dt.year==2013]
print(len(stock_2013['symbol'].unique()))

fundamentals_2013 = fundamentals[fundamentals['Period Ending'].dt.year==2013]
print(len(fundamentals_2013['Ticker Symbol'].unique()))

490
439


The two datasets do not share the same amount of unique tickers. There will be some missing data that is not shared between the two datasets

# Creating a New DataFrame with Target Variable
We will create a target variable from the stock price dataset.
* Target: percent returns approximately one year after<sup>[1](#footnote_1)</sup> their listed 10-K reporting date <sup>[2](#footnote_2)</sup>


<sub>
    <a name="footnote_1">1</a>: Companies must submit their 10-K forms within 60 days after the end of their fiscal year
    <br/>
<a name="footnote_2">2</a>: The reporting dates can be found in the column "Period Ending". The reporting date marks the end of the fiscal year for that company
    <br/>Note: the reporting dates are not the same as the actual dates the 10-Ks were submitted and filed. There could be a delay of up to 60 days
</sub>

# Merge Datasets

In [10]:
df_new = df_stock.copy()
df_new = df_new[['symbol','date','close']]
df_new.head()

,symbol,date,close
0,WLTW,2016-01-05,125.839996
1,WLTW,2016-01-06,119.980003
2,WLTW,2016-01-07,114.949997
3,WLTW,2016-01-08,116.620003
4,WLTW,2016-01-11,114.970001


In [11]:
fundamentals.sort_values(by='Period Ending', inplace=True) #pd.merge_asof requires the keys to be sorted

In [12]:
df_new.sort_values(by='date', inplace=True)

In [13]:
# 15 day leeway should be plenty to account for weekends and other days the stock market isn't open
# Can adjust timedelta to a different value if need be
tol = timedelta(15) 

merged = pd.merge_asof(left=fundamentals, right=df_new,
              left_on=['Period Ending'], right_on=['date'], 
              left_by=['Ticker Symbol'], right_by=['symbol'], 
                       direction='backward', tolerance=tol)

In [14]:
merged

,Unnamed: 0,Ticker Symbol,Period Ending,Accounts Payable,Accounts Receivable,Add'l income/expense items,After Tax ROE,Capital Expenditures,Capital Surplus,Cash Ratio,...,Total Liabilities,Total Liabilities & Equity,Total Revenue,Treasury Stock,For Year,Earnings Per Share,Estimated Shares Outstanding,symbol,date,close
0,378,COTY,2003-06-30,1.330000e+05,926000.0,19000.0,2975.0,-7.400000e+04,1.672000e+06,89.0,...,2.662000e+06,2.705000e+06,1.577000e+06,0.000000e+00,NaN,NaN,NaN,NaN,NaT,NaN
1,379,COTY,2004-06-30,2.630000e+05,275000.0,22000.0,79.0,-5.000000e+04,1.024700e+07,508.0,...,2.577000e+06,7.188000e+06,1.514000e+06,0.000000e+00,NaN,NaN,NaN,NaN,NaT,NaN
2,380,COTY,2006-02-28,1.307800e+07,1351000.0,3138000.0,552.0,-1.361000e+06,5.726200e+07,14.0,...,9.407700e+07,1.029460e+08,7.956200e+07,0.000000e+00,NaN,NaN,NaN,NaN,NaT,NaN
3,381,COTY,2007-02-28,1.556900e+07,-2979000.0,-54669000.0,134.0,-8.960000e+05,7.482800e+07,15.0,...,2.940330e+08,2.168950e+08,9.964200e+07,0.000000e+00,NaN,NaN,NaN,NaN,NaT,NaN
4,206,BBY,2012-03-03,7.876000e+09,41000000.0,77000000.0,33.0,-7.660000e+08,0.000000e+00,14.0,...,1.226000e+10,1.600500e+10,4.545700e+10,0.000000e+00,2012.0,-3.36,3.663690e+08,BBY,2012-03-02,24.309999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1776,974,LMT,2016-12-31,1.019300e+10,-811000000.0,0.0,351.0,-1.063000e+09,0.000000e+00,15.0,...,4.629500e+10,4.780600e+10,4.724800e+10,0.000000e+00,NaN,NaN,NaN,LMT,2016-12-30,249.940002
1777,994,LUV,2016-12-31,3.163000e+09,-50000000.0,-138000000.0,27.0,-2.038000e+09,1.410000e+09,48.0,...,1.484500e+10,2.328600e+10,2.042500e+10,-4.872000e+09,NaN,NaN,NaN,LUV,2016-12-30,49.840000
1778,731,HAL,2016-12-31,2.526000e+09,899000000.0,-208000000.0,61.0,-7.980000e+08,2.010000e+08,100.0,...,1.759100e+10,2.700000e+10,1.588700e+10,-7.153000e+09,NaN,NaN,NaN,HAL,2016-12-30,54.090000
1779,1780,ZTS,2016-12-31,1.076000e+09,15000000.0,2000000.0,55.0,-2.160000e+08,1.024000e+09,65.0,...,6.162000e+09,7.649000e+09,4.888000e+09,-4.210000e+08,NaN,NaN,NaN,ZTS,2016-12-30,53.529999


#  Subset the Newly Merged Data
Since, stocks do not share the same fiscal year and a company's fiscal year does not always align with the calendar year, a windowed period is helpful to subset the data. Let's take stocks with fiscal year periods in the range of 2013 or 2014. We will drop duplicate tickers in order to keep only unique tickers in the dataframe. 

In [15]:
merged_2 = merged[(merged['Period Ending'].dt.year==2013) | (merged['Period Ending'].dt.year==2014)]
merged_2

,Unnamed: 0,Ticker Symbol,Period Ending,Accounts Payable,Accounts Receivable,Add'l income/expense items,After Tax ROE,Capital Expenditures,Capital Surplus,Cash Ratio,...,Total Liabilities,Total Liabilities & Equity,Total Revenue,Treasury Stock,For Year,Earnings Per Share,Estimated Shares Outstanding,symbol,date,close
240,1184,NVDA,2013-01-27,9.762230e+08,-1.189400e+08,17094000.0,12.0,-1.833090e+08,3.193623e+09,382.0,...,1.584542e+09,6.412245e+09,4.280159e+09,-1.622709e+09,2013.0,0.91,6.181714e+08,NVDA,2013-01-25,12.410000
241,40,ADSK,2013-01-31,3.967000e+08,-9.810000e+07,0.0,12.0,-5.640000e+07,0.000000e+00,187.0,...,2.265200e+09,4.308400e+09,2.312200e+09,0.000000e+00,2013.0,1.09,2.269725e+08,ADSK,2013-01-31,38.880001
242,1706,WMT,2013-01-31,5.909900e+10,-6.140000e+08,186000000.0,22.0,-1.289800e+10,3.620000e+09,11.0,...,1.267620e+11,2.031050e+11,4.686510e+11,0.000000e+00,2013.0,5.04,3.372817e+09,WMT,2013-01-31,69.949997
243,1522,TIF,2013-01-31,3.259110e+08,-1.393000e+06,5428000.0,16.0,-2.195300e+08,1.019997e+09,86.0,...,2.032118e+09,4.630850e+09,3.794249e+09,0.000000e+00,2012.0,3.28,1.268771e+08,TIF,2013-01-31,65.750000
244,1610,URBN,2013-01-31,2.501950e+08,-2.917000e+06,1287000.0,18.0,-1.688750e+08,4.827600e+07,189.0,...,4.426230e+08,1.797211e+09,2.794925e+09,0.000000e+00,2013.0,1.63,1.455914e+08,URBN,2013-01-31,42.799999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1110,125,AN,2014-12-31,2.355400e+09,-8.030000e+07,1500000.0,20.0,-2.092000e+08,6.180000e+07,2.0,...,6.327600e+09,8.399700e+09,1.910880e+10,-1.747900e+09,2014.0,3.57,1.172829e+08,AN,2014-12-31,60.410000
1111,130,ANTM,2014-12-31,5.167100e+09,-1.899700e+09,-81100000.0,11.0,-7.146000e+08,1.006230e+10,NaN,...,3.742500e+10,6.167630e+10,7.387410e+10,0.000000e+00,2013.0,9.31,2.760150e+08,ANTM,2014-12-31,125.669998
1112,817,HSY,2014-12-31,1.300146e+09,-6.746400e+07,-2686000.0,58.0,-3.459470e+08,7.541860e+08,24.0,...,4.167808e+09,5.622870e+09,7.421768e+09,-5.161236e+09,2014.0,NaN,NaN,HSY,2014-12-31,103.930000
1113,590,EW,2014-12-31,4.261000e+08,-2.680000e+07,-1300000.0,37.0,-8.290000e+07,8.784000e+08,338.0,...,1.331600e+09,3.523000e+09,2.322900e+09,-1.556900e+09,2014.0,7.62,1.064436e+08,EW,2014-12-31,63.689999


In [16]:
# Drop duplicate tickers
merged_2 = merged_2.drop_duplicates(subset='Ticker Symbol', keep='last')
merged_2

,Unnamed: 0,Ticker Symbol,Period Ending,Accounts Payable,Accounts Receivable,Add'l income/expense items,After Tax ROE,Capital Expenditures,Capital Surplus,Cash Ratio,...,Total Liabilities,Total Liabilities & Equity,Total Revenue,Treasury Stock,For Year,Earnings Per Share,Estimated Shares Outstanding,symbol,date,close
346,892,K,2013-12-28,1.432000e+09,-5.000000e+07,4000000.0,51.0,-637000000.0,6.260000e+08,7.0,...,1.192900e+10,1.547400e+10,1.479200e+10,-2.999000e+09,2013.0,4.98,3.628514e+08,K,2013-12-27,60.980000
347,1466,SWK,2013-12-28,2.772400e+09,-5.000000e+05,-7800000.0,7.0,-340300000.0,4.878600e+09,15.0,...,9.735900e+09,1.653510e+10,1.088950e+10,-1.454400e+09,2013.0,3.16,1.486392e+08,SWK,2013-12-27,81.010002
348,5,AAP,2013-12-28,2.609239e+09,-3.242800e+07,2698000.0,26.0,-195757000.0,5.312930e+08,40.0,...,4.048569e+09,5.564774e+09,6.493814e+09,-1.078900e+08,2013.0,5.36,7.308918e+07,AAP,2013-12-27,109.919998
354,167,AVY,2013-12-28,1.477200e+09,-1.360000e+08,-36600000.0,14.0,-129200000.0,8.123000e+08,23.0,...,3.118400e+09,4.610600e+09,6.140000e+09,-1.172200e+09,2013.0,2.19,9.735160e+07,AVY,2013-12-27,50.480000
355,286,CERN,2013-12-28,3.765790e+08,-9.599000e+06,12042000.0,13.0,-352877000.0,8.128530e+08,137.0,...,9.307000e+08,4.098364e+09,2.910748e+09,-2.825100e+07,2013.0,1.16,3.434086e+08,CERN,2013-12-27,55.580002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1110,125,AN,2014-12-31,2.355400e+09,-8.030000e+07,1500000.0,20.0,-209200000.0,6.180000e+07,2.0,...,6.327600e+09,8.399700e+09,1.910880e+10,-1.747900e+09,2014.0,3.57,1.172829e+08,AN,2014-12-31,60.410000
1111,130,ANTM,2014-12-31,5.167100e+09,-1.899700e+09,-81100000.0,11.0,-714600000.0,1.006230e+10,NaN,...,3.742500e+10,6.167630e+10,7.387410e+10,0.000000e+00,2013.0,9.31,2.760150e+08,ANTM,2014-12-31,125.669998
1112,817,HSY,2014-12-31,1.300146e+09,-6.746400e+07,-2686000.0,58.0,-345947000.0,7.541860e+08,24.0,...,4.167808e+09,5.622870e+09,7.421768e+09,-5.161236e+09,2014.0,NaN,NaN,HSY,2014-12-31,103.930000
1113,590,EW,2014-12-31,4.261000e+08,-2.680000e+07,-1300000.0,37.0,-82900000.0,8.784000e+08,338.0,...,1.331600e+09,3.523000e+09,2.322900e+09,-1.556900e+09,2014.0,7.62,1.064436e+08,EW,2014-12-31,63.689999


**Check Null Values**

In [17]:
merged_2.isnull().sum().sort_values(ascending=False)[:20]

Quick Ratio                     75
Current Ratio                   75
Cash Ratio                      75
Estimated Shares Outstanding    31
Earnings Per Share              31
For Year                        18
close                            9
symbol                           9
date                             9
Intangible Assets                0
Fixed Assets                     0
Goodwill                         0
Gross Margin                     0
Gross Profit                     0
Income Tax                       0
Investments                      0
Interest Expense                 0
Inventory                        0
Effect of Exchange Rate          0
Liabilities                      0
dtype: int64

There are null values in the "symbol" column. Let's check out these rows

In [18]:
merged_2[merged_2['symbol'].isna()]

,Unnamed: 0,Ticker Symbol,Period Ending,Accounts Payable,Accounts Receivable,Add'l income/expense items,After Tax ROE,Capital Expenditures,Capital Surplus,Cash Ratio,...,Total Liabilities,Total Liabilities & Equity,Total Revenue,Treasury Stock,For Year,Earnings Per Share,Estimated Shares Outstanding,symbol,date,close
710,1335,QRVO,2014-03-29,1.316070e+08,6.160000e+06,2515000.0,2.0,-6.675300e+07,0.000000e+00,111.0,...,2.439610e+08,9.203120e+08,1.148231e+09,0.0,NaN,NaN,NaN,NaN,NaT,NaN
769,1710,WRK,2014-09-30,1.037200e+09,6.730000e+07,2400000.0,11.0,-5.342000e+08,2.839800e+09,3.0,...,6.732900e+09,1.103970e+10,9.895100e+09,0.0,NaN,NaN,NaN,NaN,NaT,NaN
780,788,HPE,2014-10-31,1.302600e+10,1.414000e+09,0.0,4.0,-3.620000e+09,0.000000e+00,12.0,...,2.829500e+10,6.507100e+10,5.512300e+10,0.0,NaN,NaN,NaN,NaN,NaT,NaN
802,1328,PYPL,2014-12-31,2.671000e+09,-3.700000e+07,-7000000.0,5.0,-4.920000e+08,0.000000e+00,17.0,...,1.366900e+10,2.191700e+10,8.025000e+09,0.0,NaN,NaN,NaN,NaN,NaT,NaN
831,1480,SYF,2014-12-31,3.469000e+09,6.800000e+07,0.0,20.0,0.000000e+00,9.408000e+09,NaN,...,6.522900e+10,7.570700e+10,1.272700e+10,0.0,2014.0,2.78,7.586331e+08,NaN,NaT,NaN
839,1130,NAVI,2014-12-31,0.000000e+00,-7.500000e+07,0.0,27.0,0.000000e+00,2.893000e+09,NaN,...,1.421540e+11,1.463520e+11,5.637000e+09,-432000000.0,2014.0,2.74,4.193431e+08,NaN,NaT,NaN
924,1572,UA,2014-12-31,3.581130e+08,-1.010570e+08,-6410000.0,15.0,-1.405280e+08,5.083500e+08,141.0,...,7.447830e+08,2.095083e+09,3.084370e+09,0.0,2014.0,0.98,2.122878e+08,NaN,NaT,NaN
947,1696,WLTW,2014-12-31,3.300000e+07,-6.600000e+07,6000000.0,18.0,-1.130000e+08,1.524000e+09,NaN,...,1.343600e+10,1.542100e+10,3.802000e+09,-3000000.0,2014.0,NaN,NaN,NaN,NaT,NaN
1005,296,CFG,2014-12-31,0.000000e+00,0.000000e+00,0.0,4.0,-1.410000e+08,1.867600e+10,NaN,...,1.135890e+11,1.328570e+11,5.342000e+09,-336000000.0,2014.0,1.55,5.580645e+08,NaN,NaT,NaN


**Let's drop all the rows that have null values in the symbol column as they do not have closing price information**

In [19]:
merged_2 = merged_2.dropna(subset=['symbol'])
merged_2['symbol'].isnull().any()

False

We will deal with the rest of the missing values later on

# Create percent returns variable
- Create new column of dates for chosen investment timeline
- Create new column with stock closing price associated with the new date column
- Calculate percent returns for each stock
- Assign percent returns values to a new variable

In [20]:
df = merged_2.copy()

In [21]:
number_days = 365
df['dates_1'] = df['date'] + timedelta(days=number_days)

In [22]:
df.head()

,Unnamed: 0,Ticker Symbol,Period Ending,Accounts Payable,Accounts Receivable,Add'l income/expense items,After Tax ROE,Capital Expenditures,Capital Surplus,Cash Ratio,...,Total Liabilities & Equity,Total Revenue,Treasury Stock,For Year,Earnings Per Share,Estimated Shares Outstanding,symbol,date,close,dates_1
346,892,K,2013-12-28,1.432000e+09,-50000000.0,4000000.0,51.0,-637000000.0,6.260000e+08,7.0,...,1.547400e+10,1.479200e+10,-2.999000e+09,2013.0,4.98,3.628514e+08,K,2013-12-27,60.980000,2014-12-27
347,1466,SWK,2013-12-28,2.772400e+09,-500000.0,-7800000.0,7.0,-340300000.0,4.878600e+09,15.0,...,1.653510e+10,1.088950e+10,-1.454400e+09,2013.0,3.16,1.486392e+08,SWK,2013-12-27,81.010002,2014-12-27
348,5,AAP,2013-12-28,2.609239e+09,-32428000.0,2698000.0,26.0,-195757000.0,5.312930e+08,40.0,...,5.564774e+09,6.493814e+09,-1.078900e+08,2013.0,5.36,7.308918e+07,AAP,2013-12-27,109.919998,2014-12-27
354,167,AVY,2013-12-28,1.477200e+09,-136000000.0,-36600000.0,14.0,-129200000.0,8.123000e+08,23.0,...,4.610600e+09,6.140000e+09,-1.172200e+09,2013.0,2.19,9.735160e+07,AVY,2013-12-27,50.480000,2014-12-27
355,286,CERN,2013-12-28,3.765790e+08,-9599000.0,12042000.0,13.0,-352877000.0,8.128530e+08,137.0,...,4.098364e+09,2.910748e+09,-2.825100e+07,2013.0,1.16,3.434086e+08,CERN,2013-12-27,55.580002,2014-12-27


In [23]:
df_new.head()

,symbol,date,close
646,SYMC,2010-01-04,18.400000
473,IFF,2010-01-04,42.009998
474,ILMN,2010-01-04,30.549999
475,INTC,2010-01-04,20.879999
476,INTU,2010-01-04,30.809999


In [24]:
df_test = pd.merge_asof(left=df, right=df_new,
              left_on=['dates_1'], right_on=['date'], 
              left_by=['symbol'], right_by=['symbol'], direction='backward', tolerance=tol)

In [25]:
df_test.head()

,Unnamed: 0,Ticker Symbol,Period Ending,Accounts Payable,Accounts Receivable,Add'l income/expense items,After Tax ROE,Capital Expenditures,Capital Surplus,Cash Ratio,...,Treasury Stock,For Year,Earnings Per Share,Estimated Shares Outstanding,symbol,date_x,close_x,dates_1,date_y,close_y
0,892,K,2013-12-28,1.432000e+09,-50000000.0,4000000.0,51.0,-637000000.0,6.260000e+08,7.0,...,-2.999000e+09,2013.0,4.98,3.628514e+08,K,2013-12-27,60.980000,2014-12-27,2014-12-26,67.010002
1,1466,SWK,2013-12-28,2.772400e+09,-500000.0,-7800000.0,7.0,-340300000.0,4.878600e+09,15.0,...,-1.454400e+09,2013.0,3.16,1.486392e+08,SWK,2013-12-27,81.010002,2014-12-27,2014-12-26,97.050003
2,5,AAP,2013-12-28,2.609239e+09,-32428000.0,2698000.0,26.0,-195757000.0,5.312930e+08,40.0,...,-1.078900e+08,2013.0,5.36,7.308918e+07,AAP,2013-12-27,109.919998,2014-12-27,2014-12-26,160.000000
3,167,AVY,2013-12-28,1.477200e+09,-136000000.0,-36600000.0,14.0,-129200000.0,8.123000e+08,23.0,...,-1.172200e+09,2013.0,2.19,9.735160e+07,AVY,2013-12-27,50.480000,2014-12-27,2014-12-26,52.619999
4,286,CERN,2013-12-28,3.765790e+08,-9599000.0,12042000.0,13.0,-352877000.0,8.128530e+08,137.0,...,-2.825100e+07,2013.0,1.16,3.434086e+08,CERN,2013-12-27,55.580002,2014-12-27,2014-12-26,65.709999


In [26]:
df_test['return'] = (df_test['close_y'] - df_test['close_x']) / df_test['close_x'] * 100

In [27]:
df_test

,Unnamed: 0,Ticker Symbol,Period Ending,Accounts Payable,Accounts Receivable,Add'l income/expense items,After Tax ROE,Capital Expenditures,Capital Surplus,Cash Ratio,...,For Year,Earnings Per Share,Estimated Shares Outstanding,symbol,date_x,close_x,dates_1,date_y,close_y,return
0,892,K,2013-12-28,1.432000e+09,-5.000000e+07,4000000.0,51.0,-637000000.0,6.260000e+08,7.0,...,2013.0,4.98,3.628514e+08,K,2013-12-27,60.980000,2014-12-27,2014-12-26,67.010002,9.888491
1,1466,SWK,2013-12-28,2.772400e+09,-5.000000e+05,-7800000.0,7.0,-340300000.0,4.878600e+09,15.0,...,2013.0,3.16,1.486392e+08,SWK,2013-12-27,81.010002,2014-12-27,2014-12-26,97.050003,19.800025
2,5,AAP,2013-12-28,2.609239e+09,-3.242800e+07,2698000.0,26.0,-195757000.0,5.312930e+08,40.0,...,2013.0,5.36,7.308918e+07,AAP,2013-12-27,109.919998,2014-12-27,2014-12-26,160.000000,45.560410
3,167,AVY,2013-12-28,1.477200e+09,-1.360000e+08,-36600000.0,14.0,-129200000.0,8.123000e+08,23.0,...,2013.0,2.19,9.735160e+07,AVY,2013-12-27,50.480000,2014-12-27,2014-12-26,52.619999,4.239301
4,286,CERN,2013-12-28,3.765790e+08,-9.599000e+06,12042000.0,13.0,-352877000.0,8.128530e+08,137.0,...,2013.0,1.16,3.434086e+08,CERN,2013-12-27,55.580002,2014-12-27,2014-12-26,65.709999,18.225975
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
431,125,AN,2014-12-31,2.355400e+09,-8.030000e+07,1500000.0,20.0,-209200000.0,6.180000e+07,2.0,...,2014.0,3.57,1.172829e+08,AN,2014-12-31,60.410000,2015-12-31,2015-12-31,59.660000,-1.241516
432,130,ANTM,2014-12-31,5.167100e+09,-1.899700e+09,-81100000.0,11.0,-714600000.0,1.006230e+10,NaN,...,2013.0,9.31,2.760150e+08,ANTM,2014-12-31,125.669998,2015-12-31,2015-12-31,139.440002,10.957272
433,817,HSY,2014-12-31,1.300146e+09,-6.746400e+07,-2686000.0,58.0,-345947000.0,7.541860e+08,24.0,...,2014.0,NaN,NaN,HSY,2014-12-31,103.930000,2015-12-31,2015-12-31,89.269997,-14.105651
434,590,EW,2014-12-31,4.261000e+08,-2.680000e+07,-1300000.0,37.0,-82900000.0,8.784000e+08,338.0,...,2014.0,7.62,1.064436e+08,EW,2014-12-31,63.689999,2015-12-31,2015-12-31,78.980003,24.006916


In [28]:
# Drop Unnecessary Columns
df_test.drop(['Unnamed: 0', 'Ticker Symbol', 'dates_1'], axis=1, inplace=True)

In [29]:
df_test.head()

,Period Ending,Accounts Payable,Accounts Receivable,Add'l income/expense items,After Tax ROE,Capital Expenditures,Capital Surplus,Cash Ratio,Cash and Cash Equivalents,Changes in Inventories,...,Treasury Stock,For Year,Earnings Per Share,Estimated Shares Outstanding,symbol,date_x,close_x,date_y,close_y,return
0,2013-12-28,1.432000e+09,-50000000.0,4000000.0,51.0,-637000000.0,6.260000e+08,7.0,2.730000e+08,112000000.0,...,-2.999000e+09,2013.0,4.98,3.628514e+08,K,2013-12-27,60.980000,2014-12-26,67.010002,9.888491
1,2013-12-28,2.772400e+09,-500000.0,-7800000.0,7.0,-340300000.0,4.878600e+09,15.0,4.962000e+08,-101900000.0,...,-1.454400e+09,2013.0,3.16,1.486392e+08,SWK,2013-12-27,81.010002,2014-12-26,97.050003,19.800025
2,2013-12-28,2.609239e+09,-32428000.0,2698000.0,26.0,-195757000.0,5.312930e+08,40.0,1.112471e+09,-203513000.0,...,-1.078900e+08,2013.0,5.36,7.308918e+07,AAP,2013-12-27,109.919998,2014-12-26,160.000000,45.560410
3,2013-12-28,1.477200e+09,-136000000.0,-36600000.0,14.0,-129200000.0,8.123000e+08,23.0,3.516000e+08,-75900000.0,...,-1.172200e+09,2013.0,2.19,9.735160e+07,AVY,2013-12-27,50.480000,2014-12-26,52.619999,4.239301
4,2013-12-28,3.765790e+08,-9599000.0,12042000.0,13.0,-352877000.0,8.128530e+08,137.0,2.023770e+08,-8111000.0,...,-2.825100e+07,2013.0,1.16,3.434086e+08,CERN,2013-12-27,55.580002,2014-12-26,65.709999,18.225975


In [30]:
# Reorganize columns in dataframe
df_test = df_test[list(df_test.columns[-6:]) + list(df_test.columns[:-6])]
df_test.head()

,symbol,date_x,close_x,date_y,close_y,return,Period Ending,Accounts Payable,Accounts Receivable,Add'l income/expense items,...,Total Current Assets,Total Current Liabilities,Total Equity,Total Liabilities,Total Liabilities & Equity,Total Revenue,Treasury Stock,For Year,Earnings Per Share,Estimated Shares Outstanding
0,K,2013-12-27,60.980000,2014-12-26,67.010002,9.888491,2013-12-28,1.432000e+09,-50000000.0,4000000.0,...,3.267000e+09,3.835000e+09,3.545000e+09,1.192900e+10,1.547400e+10,1.479200e+10,-2.999000e+09,2013.0,4.98,3.628514e+08
1,SWK,2013-12-27,81.010002,2014-12-26,97.050003,19.800025,2013-12-28,2.772400e+09,-500000.0,-7800000.0,...,4.016600e+09,3.236000e+09,6.799200e+09,9.735900e+09,1.653510e+10,1.088950e+10,-1.454400e+09,2013.0,3.16,1.486392e+08
2,AAP,2013-12-27,109.919998,2014-12-26,160.000000,45.560410,2013-12-28,2.609239e+09,-32428000.0,2698000.0,...,3.989384e+09,2.764785e+09,1.516205e+09,4.048569e+09,5.564774e+09,6.493814e+09,-1.078900e+08,2013.0,5.36,7.308918e+07
3,AVY,2013-12-27,50.480000,2014-12-26,52.619999,4.239301,2013-12-28,1.477200e+09,-136000000.0,-36600000.0,...,2.091800e+09,1.554100e+09,1.492200e+09,3.118400e+09,4.610600e+09,6.140000e+09,-1.172200e+09,2013.0,2.19,9.735160e+07
4,CERN,2013-12-27,55.580002,2014-12-26,65.709999,18.225975,2013-12-28,3.765790e+08,-9599000.0,12042000.0,...,1.761708e+09,6.404320e+08,3.167664e+09,9.307000e+08,4.098364e+09,2.910748e+09,-2.825100e+07,2013.0,1.16,3.434086e+08


In [31]:
print("Descriptive statistics of returns column:")
df_test['return'].describe()

Descriptive statistics of returns column:


count    436.000000
mean       2.182700
std       26.126130
min      -77.005621
25%      -14.184528
50%        1.731709
75%       16.541152
max      134.378373
Name: return, dtype: float64

##  Set Threshold and Create Target Variable 
Create a boolean variable using "returns" columns. If a stock has a return of at least 20%, then 1, else 0. This process will turn our business problem into a binary classifcation problem.

In [32]:
df = df_test.copy()
df['target'] = np.where(df['return']>=20, 1, 0) # sets threshold

#Check that target variable is correct
df.head()

,symbol,date_x,close_x,date_y,close_y,return,Period Ending,Accounts Payable,Accounts Receivable,Add'l income/expense items,...,Total Current Liabilities,Total Equity,Total Liabilities,Total Liabilities & Equity,Total Revenue,Treasury Stock,For Year,Earnings Per Share,Estimated Shares Outstanding,target
0,K,2013-12-27,60.980000,2014-12-26,67.010002,9.888491,2013-12-28,1.432000e+09,-50000000.0,4000000.0,...,3.835000e+09,3.545000e+09,1.192900e+10,1.547400e+10,1.479200e+10,-2.999000e+09,2013.0,4.98,3.628514e+08,0
1,SWK,2013-12-27,81.010002,2014-12-26,97.050003,19.800025,2013-12-28,2.772400e+09,-500000.0,-7800000.0,...,3.236000e+09,6.799200e+09,9.735900e+09,1.653510e+10,1.088950e+10,-1.454400e+09,2013.0,3.16,1.486392e+08,0
2,AAP,2013-12-27,109.919998,2014-12-26,160.000000,45.560410,2013-12-28,2.609239e+09,-32428000.0,2698000.0,...,2.764785e+09,1.516205e+09,4.048569e+09,5.564774e+09,6.493814e+09,-1.078900e+08,2013.0,5.36,7.308918e+07,1
3,AVY,2013-12-27,50.480000,2014-12-26,52.619999,4.239301,2013-12-28,1.477200e+09,-136000000.0,-36600000.0,...,1.554100e+09,1.492200e+09,3.118400e+09,4.610600e+09,6.140000e+09,-1.172200e+09,2013.0,2.19,9.735160e+07,0
4,CERN,2013-12-27,55.580002,2014-12-26,65.709999,18.225975,2013-12-28,3.765790e+08,-9599000.0,12042000.0,...,6.404320e+08,3.167664e+09,9.307000e+08,4.098364e+09,2.910748e+09,-2.825100e+07,2013.0,1.16,3.434086e+08,0


In [33]:
# How many 1s
df['target'].sum() / len(df['target'])

0.2018348623853211

### Save Data

In [34]:
df.to_csv("../dataset_wrangled/target.csv", index=False)